<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/TA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Datos

In [1]:
import tarfile
import numpy as np
import pandas as pd

## Recopilación

In [2]:
!wget -P /content/Data https://github.com/renato145/3D-ORGAN/blob/master/datasets/arq_dataset.tar.gz?raw=true

--2022-09-28 05:05:56--  https://github.com/renato145/3D-ORGAN/blob/master/datasets/arq_dataset.tar.gz?raw=true
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/renato145/3D-ORGAN/raw/master/datasets/arq_dataset.tar.gz [following]
--2022-09-28 05:05:57--  https://github.com/renato145/3D-ORGAN/raw/master/datasets/arq_dataset.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/renato145/3D-ORGAN/master/datasets/arq_dataset.tar.gz [following]
--2022-09-28 05:05:57--  https://raw.githubusercontent.com/renato145/3D-ORGAN/master/datasets/arq_dataset.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199

In [3]:
folder = "/content/Data/"
file_tar = folder + "arq_dataset.tar.gz?raw=true"
file_name = "custom_arq_dataset.npy"

In [4]:
file = tarfile.open(file_tar)
file.extract(file_name, folder)
file.close()

In [5]:
data = np.load(folder+file_name, allow_pickle=True)
data = data.reshape(1)

In [6]:
data

array([{'train': {'labels': ['bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'bathtub', 'ba

## A dataframe

In [7]:
train = data[0]["train"] #'labels' 'data'
train = [train["labels"],train["data"]]
#train
train[0][-1], train[1][0]

('arq', array([[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, 

In [8]:
df = pd.DataFrame(train)
df = df.T
df.columns = ['labels','data']
df

,labels,data
0,bathtub,"[[[False, False, False, False, False, False, F..."
1,bathtub,"[[[False, False, False, False, False, False, F..."
2,bathtub,"[[[False, False, False, False, False, False, F..."
3,bathtub,"[[[False, False, False, False, False, False, F..."
4,bathtub,"[[[False, False, False, False, False, False, F..."
...,...,...
4918,arq,"[[[False, False, False, False, False, False, F..."
4919,arq,"[[[False, False, False, False, False, False, F..."
4920,arq,"[[[False, False, False, False, False, False, F..."
4921,arq,"[[[False, False, False, False, False, False, F..."


## Selección de tipo de objeto

In [9]:
tipos = df['labels'].unique()
tipos

array(['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor',
       'table', 'bed', 'sofa', 'desk', 'arq'], dtype=object)

In [10]:
tipo = "sofa"  #@param ['bathtub', 'dresser', 'chair', 'night_stand', 'toilet', 'monitor', 'table', 'bed', 'sofa', 'desk', 'arq']
data_tipo = df[ df['labels']==tipo ]
data_tipo

,labels,data
3111,sofa,"[[[False, False, False, False, False, False, F..."
3112,sofa,"[[[False, False, False, False, False, False, F..."
3113,sofa,"[[[False, False, False, False, False, False, F..."
3114,sofa,"[[[False, False, False, False, False, False, F..."
3115,sofa,"[[[False, False, False, False, False, False, F..."
...,...,...
3786,sofa,"[[[False, False, False, False, False, False, F..."
3787,sofa,"[[[False, False, False, False, False, False, F..."
3788,sofa,"[[[False, False, False, False, False, False, F..."
3789,sofa,"[[[False, False, False, False, False, False, F..."


# Conversión de Binvox a STL

In [29]:
example = train[1][0]
side = 10

Para cada `True` hay 6 cuadrados de 2 triángulos c/u (12 caras por cubo)

```
Cara frontal         Cara de atrás
0,10,0 __ 10,10,0    0,10,10 __ 10,10,10
   |     /   |           |     /   |
   |   /     |           |   /     |
0,0,0 ____ 10,0,0    0,0,10 ____ 10,0,10

Cara izquierda       Cara de derecha
0,10,10 __ 0,10,0    10,10,0 __ 10,10,10
   |     /   |           |     /   |
   |   /     |           |   /     |
0,0,10 ____ 0,0,0    10,0,0 ____ 10,0,10

Cara arriba          Cara de abajo
0,10,10 __ 10,10,10  0,0,0 ____ 10,0,0
   |     /   |           |     /   |
   |   /     |           |   /     |
0,10,0 __ 10,10,0    0,0,10 ____ 10,0,10
```

In [36]:
faces = []
for z, cape in enumerate(example):
  for y, row in enumerate(cape):
    for x, vox in enumerate(row):
      if vox:
        ffront = [[[x,y,z], [x+1,y+1,z], [x,y+1,z]],
                  [[x,y,z], [x+1,y+1,z], [x+1,y,z]]]
        fback  = [[[x,y,z+1], [x+1,y+1,z+1], [x,y+1,z+1]],
                  [[x,y,z+1], [x+1,y+1,z+1], [x+1,y,z+1]]]
        fleft  = [[[x,y,z+1], [x,y+1,z], [x,y+1,z+1]],
                  [[x,y,z+1], [x,y+1,z], [x,y,z]]]
        fright = [[[x+1,y,z], [x+1,y+1,z+1], [x+1,y+1,z]],
                  [[x+1,y,z], [x+1,y+1,z+1], [x+1,y,z+1]]]
        fup    = [[[x,y+1,z], [x+1,y+1,z+1], [x,y+1,z+1]],
                  [[x,y+1,z], [x+1,y+1,z+1], [x+1,y+1,z]]]
        fdown  = [[[x,y,z+1], [x+1,y,z], [x,y,z]],
                  [[x,y,z+1], [x+1,y,z], [x+1,y,z+1]]]
        faces += [ffront, fback, fleft, fright, fup, fdown]
faces

[[[[23, 4, 8], [24, 5, 8], [23, 5, 8]], [[23, 4, 8], [24, 5, 8], [24, 4, 8]]],
 [[[23, 4, 9], [24, 5, 9], [23, 5, 9]], [[23, 4, 9], [24, 5, 9], [24, 4, 9]]],
 [[[23, 4, 9], [23, 5, 8], [23, 5, 9]], [[23, 4, 9], [23, 5, 8], [23, 4, 8]]],
 [[[24, 4, 8], [24, 5, 9], [24, 5, 8]], [[24, 4, 8], [24, 5, 9], [24, 4, 9]]],
 [[[23, 5, 8], [24, 5, 9], [23, 5, 9]], [[23, 5, 8], [24, 5, 9], [24, 5, 8]]],
 [[[23, 4, 9], [24, 4, 8], [23, 4, 8]], [[23, 4, 9], [24, 4, 8], [24, 4, 9]]],
 [[[12, 5, 8], [13, 6, 8], [12, 6, 8]], [[12, 5, 8], [13, 6, 8], [13, 5, 8]]],
 [[[12, 5, 9], [13, 6, 9], [12, 6, 9]], [[12, 5, 9], [13, 6, 9], [13, 5, 9]]],
 [[[12, 5, 9], [12, 6, 8], [12, 6, 9]], [[12, 5, 9], [12, 6, 8], [12, 5, 8]]],
 [[[13, 5, 8], [13, 6, 9], [13, 6, 8]], [[13, 5, 8], [13, 6, 9], [13, 5, 9]]],
 [[[12, 6, 8], [13, 6, 9], [12, 6, 9]], [[12, 6, 8], [13, 6, 9], [13, 6, 8]]],
 [[[12, 5, 9], [13, 5, 8], [12, 5, 8]], [[12, 5, 9], [13, 5, 8], [13, 5, 9]]],
 [[[13, 5, 8], [14, 6, 8], [13, 6, 8]], [[13, 5, 8],